In [28]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from utils import *

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

import torch
from torch.autograd import Variable
import torch.utils.data as Data

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
data = pd.read_csv('../data/data_Ea83680_k026000000000.0_V0.2.csv', index_col=0)
V = 0.2

In [3]:
x_cols = ['Cae', 'Cbe', 'T', 'Q']
y_col = ['Cc']

minmax = MinMaxScaler()
in_data = minmax.fit_transform(data[x_cols].to_numpy())

X_train, X_test, Y_train, Y_test = train_test_split(in_data, data[y_col].to_numpy(), test_size=.2, )
Y_test = Y_test.reshape(Y_test.shape[0])
Y_train = Y_train.reshape(Y_train.shape[0])

print(f'x train: {X_train.shape}, x test: {X_test.shape}')
print(f'y train: {Y_train.shape}, y test: {Y_test.shape}')

x train: (8000, 4), x test: (2000, 4)
y train: (8000,), y test: (2000,)


### Falta:
- minmax scaler
- train/test split

In [46]:
x = torch.tensor(data[x_cols].to_numpy())
y = torch.tensor(data[y_col].to_numpy())

In [53]:
net = torch.nn.Sequential(
    torch.nn.Linear(in_data.shape[1], 1),
    torch.nn.ReLU(),
    torch.nn.Linear(1, 1)
)
optmizer = torch.optim.Adam(net.parameters())
loss_func = torch.nn.MSELoss()

BATCH_SIZE = 50
EPOCH = 100
torch_dataset = Data.TensorDataset(x, y)
loader = Data.DataLoader(
    dataset=torch_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True, num_workers=2,
)


In [87]:
for epoch in range(EPOCH):
    print(epoch)
    for step, (batch_x, batch_y) in enumerate(loader): # for each training step
        b_x = Variable(batch_x).float()
        b_y = Variable(batch_y).float()
        prediction = net(b_x)
        loss = loss_func(prediction, b_y)
        optmizer.zero_grad()
        loss.backward()
        optmizer.step()

In [89]:
loss

tensor(11244.7021, grad_fn=<MseLossBackward>)